## ITVERSITY CCA-175 Practice

In [65]:
import findspark as fs
import os

data_path=os.path.dirname("D://Bigdata Tutorials//data//retail_db//")
data_path_json=os.path.dirname("D://Bigdata Tutorials//data//retail_db_json//")

fs.init()

from pyspark.sql import SparkSession,SQLContext,HiveContext
spark=SparkSession.builder.appName('ITVERSITY').master('local').getOrCreate()
sc=spark.sparkContext
sqlcontext=SQLContext(sc)
sc

<SparkContext master=local appName=ITVERSITY>

In [2]:
orderItems=sc.textFile(os.path.join(data_path,"order_items"))

In [11]:
type(orderItems)

pyspark.rdd.RDD

In [3]:
int(orderItems.first().split(",")[1])

1

In [4]:
float(orderItems.first().split(",")[4])

299.98

In [5]:
orderItemsMap=orderItems.map(lambda x:(int(x.split(",")[1]),float(x.split(",")[4])))

In [6]:
revenuePerOrder=orderItemsMap.reduceByKey(lambda agg,x: agg+x)

In [7]:
for i in revenuePerOrder.take(10): print(i)

(1, 299.98)
(2, 579.98)
(4, 699.85)
(5, 1129.8600000000001)
(7, 579.9200000000001)
(8, 729.8399999999999)
(9, 599.96)
(10, 651.9200000000001)
(11, 919.79)
(12, 1299.8700000000001)


### Transformations follow LazyEvaluation.

#### LazyEvaluation simply uses a DAG(Directed Acyclic Graph) to store all the information related to the Transformations being made.

#### As soon as an Action is 'run', spark executes the DAG first and then the action.

Let's find DAG of transformations:


In [8]:
orderItems.toDebugString()

'(1) D://Bigdata Tutorials//data//retail_db\\order_items MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []\n |  D://Bigdata Tutorials//data//retail_db\\order_items HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []'

In [9]:
orderItemsMap.toDebugString()

'(1) PythonRDD[9] at RDD at PythonRDD.scala:53 []\n |  D://Bigdata Tutorials//data//retail_db\\order_items MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []\n |  D://Bigdata Tutorials//data//retail_db\\order_items HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []'

In [10]:
revenuePerOrder.toDebugString()

'(1) PythonRDD[10] at RDD at PythonRDD.scala:53 []\n |  MapPartitionsRDD[7] at mapPartitions at PythonRDD.scala:133 []\n |  ShuffledRDD[6] at partitionBy at NativeMethodAccessorImpl.java:0 []\n +-(1) PairwiseRDD[5] at reduceByKey at <ipython-input-6-6b4951bbe08c>:1 []\n    |  PythonRDD[4] at reduceByKey at <ipython-input-6-6b4951bbe08c>:1 []\n    |  D://Bigdata Tutorials//data//retail_db\\order_items MapPartitionsRDD[1] at textFile at NativeMethodAccessorImpl.java:0 []\n    |  D://Bigdata Tutorials//data//retail_db\\order_items HadoopRDD[0] at textFile at NativeMethodAccessorImpl.java:0 []'

### Now we can open spark job UI and look into DAG visualization.


### NOTE: DO NOT USE collect() to preview data in REAL LIFE ENVIRONMENT!!!

### Another way tp create an RDD is to open a file using open and making a LIST out of it. (a collection)

In [21]:
productsRaw=open(data_path+"//products").read().splitlines()

In [22]:
type(productsRaw)

list

### Now, we can create an RDD using Parallelize from the collection.

In [23]:
productsRaw=sc.parallelize(productsRaw)

In [24]:
type(productsRaw)

pyspark.rdd.RDD

In [27]:
print(productsRaw.first())
type(productsRaw.first())

1,2,Quest Q64 10 FT. x 10 FT. Slant Leg Instant U,,59.98,http://images.acmesports.sports/Quest+Q64+10+FT.+x+10+FT.+Slant+Leg+Instant+Up+Canopy


str

## DATA FRAME

#### --provided by sqlContext

#### Now we can try loading multiple file formats.

In [70]:
sqlcontext.read?

##same as spaek.read?

Type:        property
String form: <property object at 0x00000000065DC4A8>
Docstring:  
Returns a :class:`DataFrameReader` that can be used to read data
in as a :class:`DataFrame`.

:return: :class:`DataFrameReader`

.. versionadded:: 1.4


In [68]:
df = spark.read.json(data_path_json+"//order_items")

In [69]:
df.show(5)

+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|order_item_id|order_item_order_id|order_item_product_id|order_item_product_price|order_item_quantity|order_item_subtotal|
+-------------+-------------------+---------------------+------------------------+-------------------+-------------------+
|            1|                  1|                  957|                  299.98|                  1|             299.98|
|            2|                  2|                 1073|                  199.99|                  1|             199.99|
|            3|                  2|                  502|                    50.0|                  5|              250.0|
|            4|                  2|                  403|                  129.99|                  1|             129.99|
|            5|                  4|                  897|                   24.99|                  2|              49.98|
+-------------+-